# 🍏 피트니스 재미: Azure Functions + AI Agent 자습서 🍎

이 자습서에서는 **Azure AI Foundry** SDK(`azure-ai-projects`, `azure-ai-inference`, `azure-ai-evaluation`, `opentelemetry-sdk` 및 `azure-core-tracing-opentelemetry`)와 **Azure Functions**를 사용하는 방법을 살펴봅니다. 방법을 보여드리겠습니다:

1. 스토리지 큐에서 수신 대기하는 Azure 함수를 **설정**합니다.
2. 이 함수를 호출할 수 있는 AI 에이전트를 **생성**합니다.
3. 에이전트에 프롬프트를 **보내면** 에이전트가 Azure 함수를 호출합니다.
4. 출력 대기열에서 처리된 결과를 **검색**합니다.

모두 건강과 피트니스를 주제로 한 재미있는 예제입니다! 멋진 예제는 계속 이어가겠지만 기억하세요:

### ⚠️ 중요 고지 사항
> **이 예제는 데모용으로만 제공되며 실제 의료 또는 건강 관련 조언을 제공하지 않습니다.** 실제 의료 또는 피트니스 관련 조언은 항상 전문가와 상담하세요.


## 전제 조건
1. Azure 구독.
2. **Azure AI Foundry** 프로젝트. (`PROJECT_CONNECTION_STRING` 및 `MODEL_DEPLOYMENT_NAME`이 필요함).
3. **Azure Functions** 환경 또는 로컬 에뮬레이터(Azurite) 및 스토리지 큐에 대한 지식.
4. `azure-ai-projects`, `azure-identity`, `opentelemetry-sdk`, `azure-core-tracing-opentelemetry` 이 설치된 **Python 3.8+**.

## 개요
대략적인 이벤트 순서를 살펴보겠습니다:

1. **입력 대기열**에서 메시지를 읽고 **출력 대기열**에 응답을 쓰도록 **Azure Function**을 설정합니다.
2. 이 큐를 참조하는 `AzureFunctionTool`을 사용하여 **AI 에이전트**를 생성합니다.
3. **사용자**가 질문 또는 명령을 제공하면 에이전트가 함수를 호출할지 여부를 결정합니다.
4. 에이전트가 **입력 대기열**에 메시지를 보내면 함수가 트리거됩니다.
5. **Azure Function**이 메시지를 처리하고 **출력 대기열**로 응답을 보냅니다.
6. 에이전트가 출력 대기열에서 응답을 가져옵니다.
7. **사용자**가 에이전트의 최종 응답을 확인합니다.

<img src="./seq-diagrams/6-az-function.png" width="30%"/>


## 1. Azure Function 설정 (예제)
다음은 **입력 대기열**에서 메시지를 수신하고 **출력 대기열**에 결과를 게시하는 Azure 함수를 구현하는 방법을 보여주는 스니펫입니다.

이 코드를 로컬 또는 클라우드 Azure 함수 환경에 맞게 조정할 수 있습니다. 함수의 실제 로직은 무엇이든 될 수 있습니다. 데모를 위해 코믹한 "foo-based" 답변이나 우스꽝스러운 "피트니스 조언" 스니펫을 반환하는 것으로 가정해 보겠습니다. 


```python
# This code might live in your Azure Functions project in a file named: __init__.py
# or similar.
import os
import json
import logging
import azure.functions as func
from azure.storage.queue import QueueClient
from azure.core.pipeline.policies import BinaryBase64EncodePolicy, BinaryBase64DecodePolicy
from azure.identity import DefaultAzureCredential

app = func.FunctionApp()

@app.function_name(name="FooReply")
@app.queue_trigger(
    arg_name="inmsg",
    queue_name="azure-function-foo-input",
    connection="STORAGE_SERVICE_ENDPOINT"  # or connection string setting name
)
def run_foo(inmsg: func.QueueMessage) -> None:
    logging.info("Azure Function triggered with a queue item.")

    # This is the queue for output
    out_queue = QueueClient(
        os.environ["STORAGE_SERVICE_ENDPOINT"],  # or read from config
        queue_name="azure-function-tool-output",
        credential=DefaultAzureCredential(),
        message_encode_policy=BinaryBase64EncodePolicy(),
        message_decode_policy=BinaryBase64DecodePolicy()
    )

    # Parse the function call payload, e.g. { "query": "Hello?", "outputqueueuri":"..."}
    payload = json.loads(inmsg.get_body().decode('utf-8'))
    user_query = payload.get("query", "")

    # Example: We'll return a comedic 'Foo says: <some witty line>'
    result_message = {
        "FooReply": f"This is Foo, responding to: {user_query}! Stay strong 💪!",
        "CorrelationId": payload.get("CorrelationId", "")
    }

    # Put the result on the output queue
    out_queue.send_message(json.dumps(result_message).encode('utf-8'))
    logging.info(f"Sent message: {result_message}")
```

### 참고
- 입력 큐 이름 `azure-function-foo-input`.
- 출력 큐 이름 `azure-function-tool-output`.
- 큐 저장소 엔드포인트에는 `STORAGE_SERVICE_ENDPOINT` 환경변수를 사용했습니다.

## 2. 노트북 설정
이제 이 노트북 환경으로 다시 전환해 보겠습니다:
1. 라이브러리를 가져옵니다.
2. `AIProjectClient`를 초기화합니다.
3. Azure Function 도구 정의 및 에이전트를 만듭니다.

In [ ]:
# We'll do our standard imports
import os
import time
from pathlib import Path
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureFunctionTool, AzureFunctionStorageQueue, MessageRole

# Load env variables from .env in parent dir
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Create AI Project Client
try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(exclude_managed_identity_credential=True, exclude_environment_credential=True),
        conn_str=os.environ["PROJECT_CONNECTION_STRING"],
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing AIProjectClient: {e}")

### Azure Function 도구로 에이전트 만들기
함수 이름(샘플의 `foo` 또는 `FooReply`)과 입력 + 출력 큐를 참조하는 도구를 정의하겠습니다. 이 예제에서는 환경변수 `STORAGE_SERVICE_ENDPOINT`에 큐 엔드포인트를 저장합니다.

고유한 명명 체계에 맞게 조정할 수 있습니다. 에이전트가 특정 키워드를 볼 때마다 이 함수를 사용하도록 지시하거나, 그냥 자체적으로 함수를 호출하도록 할 수도 있습니다.

In [ ]:
try:
    storage_endpoint = os.environ["STORAGE_SERVICE_ENDPONT"]  # Notice it's spelled STORAGE_SERVICE_ENDPONT in sample
except KeyError:
    print("❌ Please ensure STORAGE_SERVICE_ENDPONT is set in your environment.")
    storage_endpoint = None

agent = None
if storage_endpoint:
    # Create the AzureFunctionTool object
    azure_function_tool = AzureFunctionTool(
        name="foo",
        description="Get comedic or silly advice from 'Foo'.",
        parameters={
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "The question to ask Foo."},
                "outputqueueuri": {"type": "string", "description": "The output queue URI."}
            },
        },
        input_queue=AzureFunctionStorageQueue(
            queue_name="azure-function-foo-input",
            storage_service_endpoint=storage_endpoint,
        ),
        output_queue=AzureFunctionStorageQueue(
            queue_name="azure-function-tool-output",
            storage_service_endpoint=storage_endpoint,
        ),
    )

    # Construct the agent with the function tool attached
    with project_client:
        agent = project_client.agents.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"],
            name="azure-function-agent-foo",
            instructions=(
                "You are a helpful health and fitness support agent.\n" 
                "If the user says 'What would foo say?' then call the foo function.\n" 
                "Always specify the outputqueueuri as '" + storage_endpoint + "/azure-function-tool-output'.\n"
                "Respond with 'Foo says: <response>' after the tool call."
            ),
            tools=azure_function_tool.definitions,
        )
    print(f"🎉 Created agent, agent ID: {agent.id}")
else:
    print("Skipping agent creation, no storage_endpoint.")

## 3. 에이전트 테스트하기
이제 함수 호출을 트리거하는 사용자 메시지를 시뮬레이션해 보겠습니다. 대화 **thread**를 만들고, "What would foo say?"라는 사용자 질문을 게시한 다음 에이전트를 실행합니다. 

에이전트 서비스는 `azure-function-foo-input` 대기열에 메시지를 놓습니다. 함수가 이를 처리하고 `azure-function-tool-output`에 응답을 놓습니다. 에이전트가 이를 자동으로 선택하여 최종 응답을 생성합니다.


In [ ]:
def run_foo_question(user_question: str, agent_id: str):
    # 1) Create a new thread
    thread = project_client.agents.create_thread()
    print(f"📝 Created thread, thread ID: {thread.id}")

    # 2) Create a user message
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=user_question
    )
    print(f"💬 Created user message, ID: {message.id}")

    # 3) Create and process agent run
    run = project_client.agents.create_and_process_run(
        thread_id=thread.id,
        assistant_id=agent_id
    )
    print(f"🤖 Run finished with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # 4) Retrieve messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print("\n🗣️ Conversation:")
    for m in reversed(messages.data):  # oldest first
        msg_str = ""
        if m.content:
            msg_str = m.content[-1].text.value if len(m.content) > 0 else ""
        print(f"{m.role.upper()}: {msg_str}\n")

    return thread, run

# If the agent was created, let's test it!
if agent:
    my_thread, my_run = run_foo_question(
        user_question="What is the best post-workout snack? What would foo say?",
        agent_id=agent.id
    )

## 4. 정리
완료되면 에이전트를 제거합니다. 실제 시나리오에서는 에이전트를 반복적으로 사용하기 위해 보관할 수도 있습니다.


In [ ]:
if agent:
    try:
        project_client.agents.delete_agent(agent.id)
        print(f"🗑️ Deleted agent: {agent.name}")
    except Exception as e:
        print(f"❌ Error deleting agent: {e}")

# 🎉 축하합니다!
방금 **Azure Function**과 **AI 에이전트 서비스**를 결합하여 동적 큐 기반 워크플로를 만드는 방법을 살펴봤습니다. 이 재미있는 예제에서는 함수가 "Foo says..."라는 코믹한 대사를 반환했지만 실제 애플리케이션에서는 Azure 함수의 강력한 기능을 활용하여 **데이터베이스 조회**부터 **복잡한 계산**까지 모든 작업을 실행하여 결과를 AI 에이전트에게 원활하게 반환할 수 있습니다.

## 다음 단계
- **OpenTelemetry를 추가**하여 함수와 에이전트 전반에서 엔드투엔드 추적을 수집합니다.
- **평가** 파이프라인을 `azure-ai-evaluation`과 통합하여 에이전트 + 함수 워크플로가 사용자 쿼리를 얼마나 잘 처리하는지 측정하세요.
- Azure function에서 **병렬 함수 호출** 또는 고급 로직을 살펴보세요.

즐겁게 코딩하고 건강을 유지하세요! 🤸